# One Max Problem

##### One Max Problem — DEAP 1.3.1 documentation. (2020). Retrieved 11 August 2020, from https://deap.readthedocs.io/en/master/examples/ga_onemax.html

Utilizaremos la librería de Python DEAP que es un framework de cómputo evolutivo para hacer prototipos rápidamente y probar ideas. Busca hacer algoritmos explícitos y estructuras de datos transparentes. Trabaja en perfecta armonía con mecanismos de paralelismo como `multiprocessing` y `SCOOP`.

## Ejemplo que maximiza la suma de una lista de enteros.

Por ejemplo se comienza con un conjunto de listas de 0s y 1s aleatorios (e.g. [0,1,0,0,0]), y se busca evolucionar hasta que algún elemento llegue a ser de la forma [1,1,1,1,1].

Hay muchos parámetros que se pueden ajustar como la logitud de las listas, el tamaño inicial de la población, etc.

El problema a resolver ayuda a entender algunas de las posibilidades de este framework e ilustra de manera concisa el concepto de máquinas evolutivas en general. Primero importaremos los módulos necesarios



In [1]:
import random

from deap import base
from deap import creator
from deap import tools

## Creator 

Debido a que la estructura exacta de los individuos en los algoritmos genéticos depende fuertemente en la problema que se quiere resolver, DEAP no tiene una estructura explícita. Lo que si hace es darnos una manera conveniente para crear "contenedores" de atributos, asociarlos con su aptitud (fitness) llamado $\texttt{deap.creator}$. 

$\texttt{creator}$ es una fábrica de clases que las va generando durante el tiempo de ejecución. El primer argumento es el nombre de la clase, seguido de la clase base de la cual heredará, y finalmente cualquier argumento que queramos pasar a la hora de instanciar la misma.

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

## Toolbox

Utilizaremos la clase a la medida que acamos de crear y haremos uso de Toolbox para crear a nuestro individuos así como a toda la población.

Herramientas para crear a los individuos ($\texttt{random.randint}$), los individuos mismos y la población, quedará registarada bajo un contenedor llamado $\texttt{Toolbox}$. Tiene dos métodos para agregar o remover contenido, register() y unregister().

Es importante mencionar que de forma análoga a $\texttt{creator.create}$, el primer argumento es el nombre de nuestra función, luego la función que encapsula y finalmente los argumentos de la función. Al registrar las herramientas en el toolbox, solamente crea alias a la funciones ya existentes y congela parte de sus argumentos. Con esto podemos dejar algunos de los argumentos abiertos para modificarlos después. Por ejemplo, en el caso de la población.

In [3]:
toolbox = base.Toolbox()

# Generador de atributos 
#                      'attr_bool' es un atributo ('gen')
#                      que corresponde a enteros uniformemente muestreados
#                      del rango [0,1] (i.e. 0 o 1 con igual
#                      probabilidad)
toolbox.register("attr_bool", random.randint, 0, 1)

# Inicializador de estructura
#                         define 'individuo' como uno consistiendo 
#                         de 50 elementos 'attr_bool' ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 50)

# Define la población como un conjunto de individuos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# La funcion objetivo ('fitness') a maximizar
def evalOneMax(individual):
    return sum(individual),

#----------
# Registro de operaciones
#----------
# Registra el objetivo / fitness
toolbox.register("evaluate", evalOneMax)

# Registra el operador de cruza
toolbox.register("mate", tools.cxTwoPoint)

# Registra el operador de mutación que cambia cada
# atributo/gen con probabilidad of 0.0555
toolbox.register("mutate", tools.mutFlipBit, indpb=0.0555)

# operador para seleccionar individuos por el método de la ruleta
# donde cada elemento de la nueva generación se sustituye por 
# otro de la población total. La probabilidad de obtener un 1
# está dada por la suma de todos los 1s entre el total. 
# Evidentemente lo mismo para el 0. 
toolbox.register("select", tools.selTournament,fit_attr='fitness',tournsize=3)

#----------


## Evolucionando la población

Una vez que se eligen los operadores genéticos, vamos a definir un algoritmo que combine todas las partes individuales y que realice la evolución de nuestra población hasta que el problema de maximización de un parámetro se resuelva. Por convención, esto se hará dentro de una función llamada $\texttt{main()}$.

In [4]:
import logging
import timeit
# Cambia logging.INFO para obtener menos verborrea
# Cambia logging.WARNING para silenciar por completo
logger = logging.getLogger()
logger.setLevel(logging.WARNING)
def main(
    poblacion_inicial: int = 50,
    tamaño_individuo: int = 30,
    prob_cruza: float = 0.7,
    prob_mutación: float = 0.0555,
    max_iter: int = 1000,
):
    random.seed()

    # Decidí registrar al individuo y a la población aquí
    # para poder invocar la función main con todos los
    # argumentos sin tener que ir más arriba a redefinir valores
    toolbox.register(
        "individual",
        tools.initRepeat,
        creator.Individual,
        toolbox.attr_bool,
        tamaño_individuo,
    )

    # Se define la población como un conjunto de n=poblacion_inicial listas
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    # Crea una población inicial de n individuos
    # (cada uno es una lista de 0s y 1s)
    pop = toolbox.population(n=poblacion_inicial)

    # CXPB  es la probabilidad de cruza
    #
    #
    # MUTPB es la probabilidad de mutación
    CXPB, MUTPB = prob_cruza, prob_mutación

    logging.debug("Start of evolution")

    # Se evalua la aptitud (fitness) de la población entera 
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    logging.debug("  Evaluated %i individuals" % len(pop))

    # Se extrae la aptitud (fitness)
    fits = [ind.fitness.values[0] for ind in pop]

    # Esta variable guarda el número de generaciones    
    g = 0

    # Comienza la evolución
    while max(fits) < tamaño_individuo and g < max_iter:
        # Una nueva generación
        g = g + 1
        logging.debug("-- Generation %i --" % g)

        # Selecciona la nueva generación de individuos
        offspring = toolbox.select(pop, len(pop))
        # Clona los individuos seleccionados
        offspring = list(map(toolbox.clone, offspring))

        # Aplica la cruza y mutación
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cruza a dos individuos con probabilidad CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # aptitudes de los hijos
                # deben ser recalculados
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # muta con probabilidad MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evalua a los individuos con una aptitud inválida
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        logging.debug("  Evaluated %i individuals" % len(invalid_ind))

        # La población es reemplazada por los hijos
        pop[:] = offspring

        # Junta todas las aptitudes en una lista e imprime estadísticas
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5

        logging.debug("  Min %s" % min(fits))
        logging.debug("  Max %s" % max(fits))
        logging.debug("  Avg %s" % mean)
        logging.debug("  Std %s" % std)

    logging.debug("------------- Fin de la evolución actual--------------")

    best_ind = tools.selBest(pop, 1)[0]
    logging.info("Número de generaciones %s" % g)
    logging.info("El mejor individuo es %s, %s" % (best_ind, best_ind.fitness.values))
    return g





In [5]:
if __name__ == "__main__":
    main()

# Ejemplo explícito de lo que se está logrando con este algoritmo

Profesor, para no tratar de llenar con muchos datos el resultado obtenido, decidí silenciar la salida de datos. Las gráficas de hasta abajo simplemente son 100 corridas del algoritmo con diferentes configuraciones de probabilidad de mutación, tamaño de los individuos, tipo de algoritmo de selección, etc. 

Lo unico que cambié fue `logger.setLevel(logging.DEBUG)` para mostrar las evoluciones generación a generación para de una de esas 100 corridas.

# Aleatoridad y tiempo de cómputo - Torneo
 
A continuación veremos como el número de generaciones es proporcional al tiempo de cómputo. El resultado es bastante intuitivo, entre más generaciones tardamos en llegar a la lista de 1s, mayor es el tiempo de procesamiento

In [6]:
# Funcion para almacenar el número de generaciones y el tiempo de ejecución
import io
from contextlib import redirect_stdout


def generaciones_y_tiempo(
    poblacion_inicial: int = 50,
    tamaño_individuo: int = 30,
    prob_cruza: float = 0.7,
    prob_mutación: float = 0.0555,
    max_iter: int =1000):
    f = io.StringIO()
    with redirect_stdout(f):
        print(
            timeit.timeit(
                f"print(main(poblacion_inicial={poblacion_inicial} ,tamaño_individuo={tamaño_individuo} ,prob_cruza={prob_cruza},prob_mutación={prob_mutación},max_iter={max_iter}))",
                number=1,
                globals=globals(),
            )
        )
    out = f.getvalue()
    return list(out.split("\n")[:-1])


# Vamos a hacer 100 simulaciones independientes, donde llegaremos al número requerido de 30 unos e imprimiermos todo a pantalla para mostrar que efectivamente se llega al resultado.  

## Cada simulación termina al llegar al siguiente estado
```
DEBUG:root:------------- Fin de la evolución actual--------------
INFO:root:Número de generaciones 14
INFO:root:El mejor individuo es [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (30.0,)
```

In [7]:
logger.setLevel(logging.DEBUG)
datos_tarea=[generaciones_y_tiempo() for i in range(1)]

DEBUG:root:Start of evolution
DEBUG:root:  Evaluated 50 individuals
DEBUG:root:-- Generation 1 --
DEBUG:root:  Evaluated 33 individuals
DEBUG:root:  Min 14.0
DEBUG:root:  Max 21.0
DEBUG:root:  Avg 17.24
DEBUG:root:  Std 1.817250670656097
DEBUG:root:-- Generation 2 --
DEBUG:root:  Evaluated 40 individuals
DEBUG:root:  Min 14.0
DEBUG:root:  Max 23.0
DEBUG:root:  Avg 19.14
DEBUG:root:  Std 1.8440173534975182
DEBUG:root:-- Generation 3 --
DEBUG:root:  Evaluated 28 individuals
DEBUG:root:  Min 17.0
DEBUG:root:  Max 26.0
DEBUG:root:  Avg 20.5
DEBUG:root:  Std 1.5905973720586781
DEBUG:root:-- Generation 4 --
DEBUG:root:  Evaluated 38 individuals
DEBUG:root:  Min 17.0
DEBUG:root:  Max 26.0
DEBUG:root:  Avg 21.72
DEBUG:root:  Std 1.8334666618185527
DEBUG:root:-- Generation 5 --
DEBUG:root:  Evaluated 35 individuals
DEBUG:root:  Min 19.0
DEBUG:root:  Max 28.0
DEBUG:root:  Avg 23.34
DEBUG:root:  Std 1.893251171926238
DEBUG:root:-- Generation 6 --
DEBUG:root:  Evaluated 34 individuals
DEBUG:root: 

In [8]:
import pandas as pd
def limpiar_dataframe(datos:list):
    df = pd.DataFrame(datos)
    df.rename(columns={0:"Generations",1:"Compute_time"},inplace=True)
    df=df.sort_values(by="Compute_time")
    df=df.reset_index(drop=True)
    df=df.astype({"Compute_time":float, "Generations":int})
    return df
df=limpiar_dataframe(datos_tarea)
df.head()

,Generations,Compute_time
0,9,0.061121
